In [ ]:
import pandas as pd

In [ ]:
%pip install openpyxl

In [ ]:
import openpyxl
print(openpyxl.__version__)


In [ ]:
df=pd.read_excel(r"F:\project_2\Copper_Set.xlsx")
df.head(5)

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
df.rename(columns={"quantity tons":"quantity_tons"},inplace=True)
df.rename(columns={"delivery date":"delivery_date"},inplace=True)
df.rename(columns={"item type":"item_type"},inplace=True)



In [ ]:
df["item_type"]

In [ ]:
df["quantity_tons"]

In [ ]:
df["quantity_tons"]=pd.to_numeric(df["quantity_tons"],errors="coerce")
#DATA CLEANING #PREPROCESSING
   #DATA TYPE CHANGE
df["status"]=df["status"].astype('category')
df["item_type"]=df["item_type"].astype('category')
df["product_ref"]=df["product_ref"].astype('category')



In [ ]:
#DATA TYPE CONVERTION
df["item_date"]=df["item_date"].astype("Int64")
df["item_date"]=df["item_date"].astype("string")
df["item_date"]=pd.to_datetime(df["item_date"],format="%Y%m%d",errors="coerce")
#print(df["item_date"].dtypes)
df["delivery_date"]=df["delivery_date"].astype("Int64").astype("string")
df["delivery_date"]=pd.to_datetime(df["delivery_date"],format="%Y%m%d",errors="coerce")


In [ ]:
# Which values are non-numeric?

In [ ]:
df

In [ ]:
df["quantity tons"]=pd.to_numeric(df["quantity tons"],errors="coerce")

In [ ]:
null_quantity=df[pd.to_numeric(df["quantity tons"],errors="coerce").isna()]
null_quantity

In [ ]:
df["quantity tons"].isnull().sum()

In [ ]:
df["quantity_tons"].isnull().sum()

In [ ]:
#df=df.drop(null_quantity.index)

In [ ]:
#FILL MISSING VALUES
mode_country=(df["country"].mode()[0])
df["country"].fillna(mode_country,inplace=True)
df["item_date"].fillna(method="ffill", inplace=True)
df["delivery_date"].fillna(method="bfill", inplace=True)
df["application"].fillna(df["application"].mode()[0],inplace=True)

In [ ]:
num_wrong = (df['delivery_date'] < df['item_date']).sum()
print("Wrong delivery dates after correction:", num_wrong)


In [ ]:
mask=df["delivery_date"]<df["item_date"]
df.loc[mask,"delivery_date"]=df.loc[mask,"item_date"]+pd.Timedelta(days=7)

In [ ]:
df["material_ref"].nunique()

In [ ]:
#DROP NULL VALUES
df=df[df["status"].isin(["Won","Lost"])]
#print(df["country"].mode())
df=df.dropna(subset="quantity_tons")
df = df.dropna(subset=["thickness"])

#mode_country

#df["material_ref"]=df.groupby("product_ref")["material_ref"].transform
#(lambda x:x.fillna(x.mode()[0]if not x.mode().empty else "unknown" ))
df=df.drop(columns=["material_ref"])
df=df.dropna(subset=["id"])

df=df.dropna(subset=["customer","selling_price"])
df=df[df["selling_price"]>0]
df=df[df["quantity_tons"] >=0]
df=df[df["width"]>=30].copy()

In [ ]:
country_list=sorted(df["country"].unique())
country_list

In [ ]:
#pickling country list for used in the streamlit to selcet user input
import pickle
with open("country_list_v2.pkl","wb")as f:
    pickle.dump(country_list,f)

In [ ]:
application_list=sorted(df["application"].unique())

In [ ]:
application_list


In [ ]:
with open("application_list_v2.pkl","wb")as f:
    pickle.dump(application_list,f)

In [ ]:
item_list=sorted(df["item_type"].unique())

In [ ]:
item_list

In [ ]:
with open("item_type_v2.pkl","wb")as f:
    pickle.dump(item_list,f)

In [ ]:
df["product_ref"].isna().sum()

In [ ]:
product_ref_list=sorted(df["product_ref"].unique())

In [ ]:
with open("product_ref_v2.pkl","wb")as f:
    pickle.dump(product_ref_list,f)

In [ ]:
df

In [ ]:
#RESET THE INDEX
df=df.reset_index(drop=True)

In [ ]:
df

In [ ]:
df["quantity tons"].max()

In [ ]:
df.isnull().sum()

In [ ]:
num_wrong = (df['delivery date'] < df['item_date']).sum()
print("Wrong delivery dates after correction:", num_wrong)


In [ ]:
mask=df["delivery date"]<df["item_date"]
mask

In [ ]:
df.loc[mask,"delivery date"]

In [ ]:
mask=df["delivery date"]<df["item_date"]
df.loc[mask,"delivery date"]=df.loc[mask,"item_date"]+pd.Timedelta(days=7)

In [ ]:
num_wrong = (df['delivery date'] < df['item_date']).sum()
print("Wrong delivery dates after correction:", num_wrong)


In [ ]:
print((df["selling_price"]<0).sum())



In [ ]:
df=df[df["selling_price"]>0]

In [ ]:
print((df["selling_price"]<0).sum())



In [ ]:
print((df["quantity tons"]<0).sum())
df=df[df["quantity tons"] >=0]

In [ ]:
df=df[df["quantity tons"] >=0]

In [ ]:
print((df["quantity tons"]<0).sum())

In [ ]:
df["quantity tons"].dtype

In [ ]:
#outlier treatment 
import seaborn as sns
import matplotlib.pyplot as plt
sns.boxplot(x="quantity_tons",data=df)
plt.show()

In [ ]:
Q1 = df['quantity_tons'].quantile(0.25)
Q3 = df['quantity_tons'].quantile(0.75)
IQR = Q3 - Q1

q_lower_limit = Q1 - 1.5 * IQR
q_upper_limit = Q3 + 1.5 * IQR

outliers = df[(df['quantity_tons'] < q_lower_limit) | (df['quantity_tons'] > q_upper_limit)]
outliers.shape


In [ ]:
print(q_lower_limit,q_upper_limit)

In [ ]:
#capping for quatity tons
import numpy as np
df["quantity_tons"]=np.where(df["quantity_tons"]>q_upper_limit,q_upper_limit,
                    np.where(df["quantity_tons"]<q_lower_limit,q_lower_limit,df["quantity_tons"]))

In [ ]:
#outlier treatment 

import seaborn as sns
import matplotlib.pyplot as plt
sns.boxplot(x="quantity_tons",data=df)
plt.show()

In [ ]:
sns.boxplot(x=df["thickness"],data=df)
plt.show()

In [ ]:
Q1 = df['thickness'].quantile(0.25)
Q3 = df['thickness'].quantile(0.75)
IQR = Q3 - Q1

t_lower_limit = Q1 - 1.5 * IQR
t_upper_limit = Q3 + 1.5 * IQR

outliers = df[(df['thickness'] < t_lower_limit) | (df['thickness'] > t_upper_limit)]
outliers.shape


In [ ]:
print(t_lower_limit, t_upper_limit)

In [ ]:
#negative lower lmit doesnot affect the original values 
#lower_limit = max(0, lower_limit)  # set physical minimum
df['thickness'] = df['thickness'].clip(lower=t_lower_limit, upper=t_upper_limit)


In [ ]:
sns.boxplot(x=df["thickness"],data=df)
plt.show()

In [ ]:
df["thickness"].min()

In [ ]:
df["width"].min()

In [ ]:
sns.boxplot(x="width",data=df)

In [ ]:
#width is muti modal distribution  
# Width columnக்கு IQR use பண்ணக்கூடாதது இதற்குத்தான்.
#இது real industrial size variation — error இல்லை. 30 mm to 3000mm width is normal for copper
#so we remove lessthan 30 mm neg_values
df=df[df["width"]>=30].copy()
# no need to do out lier treatment

In [ ]:
print((df["width"]<=30).sum())

In [ ]:
df.describe()

In [ ]:
sns.boxplot(x="selling_price",data=df)

In [ ]:
df["selling_price"].min()

In [ ]:
Q1 = df['selling_price'].quantile(0.25)
Q3 = df['selling_price'].quantile(0.75)
IQR = Q3 - Q1

lower_limit = Q1 - 1.5 * IQR
upper_limit = Q3 + 1.5 * IQR

outliers = df[(df['selling_price'] < lower_limit) | (df['selling_price'] > upper_limit)]
outliers.shape


In [ ]:
print(lower_limit, upper_limit)

In [ ]:
#  manual capping for selling price

import numpy as np
df["selling_price"]=np.where(df["selling_price"]>upper_limit,upper_limit,
                    np.where(df["selling_price"]<lower_limit,lower_limit,df["selling_price"]))

In [ ]:
sns.boxplot(x="selling_price",data=df)

In [ ]:
#RESET THE INDEX AFTER THE OUTLIER TREATMENT
df = df.reset_index(drop=True)


In [ ]:
df

In [ ]:
%pip install scikit-learn


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df["delivery_days"]=(df["delivery_date"]-df["item_date"]).dt.days
num_col=["quantity_tons","thickness","delivery_days"]
#log transform
log_trans=np.log1p(df[num_col])
log_trans
num_scalled= scaler.fit_transform(log_trans)
num_scalled
#num_col
num_df=pd.DataFrame(num_scalled,columns=num_col)

In [ ]:
#pickle scaller object
import pickle
with open("num_scaller_v2.pkl","wb")as f:
    pickle.dump(scaler,f)

In [ ]:
df.info()

In [ ]:
from sklearn.preprocessing import RobustScaler
width_scaler=RobustScaler()
df["width_scaled"]=width_scaler.fit_transform(df[["width"]])

df["width_scaled"]

In [ ]:
#pickle width scaler object
with open("width_scaler_v2.pkl","wb")as f:
    pickle.dump(width_scaler,f)

In [ ]:
num_df["width_scaled"]=df["width_scaled"].values

In [ ]:
num_df

In [ ]:
#categorical column need to be encode here im using OHE
from sklearn.preprocessing import OneHotEncoder
df["item type"]=df["item_type"].str.upper()
#df["country"]=df["country"].astype(str)
cat_cols=["country","application","item_type","product_ref"]
ohe_encoding=OneHotEncoder(sparse_output=False, drop=None,handle_unknown="ignore")
ohe_array=ohe_encoding.fit_transform(df[cat_cols])


In [ ]:
with open("cat_encoding_v2.pkl","wb")as f:
    pickle.dump(ohe_encoding,f)

In [ ]:
ohe_array

In [ ]:
ohe_cols=ohe_encoding.get_feature_names_out(cat_cols)
#ohe_df=pd.DataFrame()

In [ ]:
num_df

In [ ]:
ohe_df=pd.DataFrame(ohe_array,columns=ohe_cols,index=df.index)
ohe_df

In [ ]:
df["selling_price"].isna().sum()

In [ ]:
df_final=pd.concat([num_df,ohe_df],axis=1)

In [ ]:
df_final

In [ ]:
df_final.isna().sum().sum()

In [ ]:
x.shape

In [ ]:
y.shape

In [ ]:
from sklearn .model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor

x=df_final
y=df["selling_price"]
X_train,X_test,Y_train,Y_test=train_test_split(
    x,y,test_size=0.2,random_state=42)

model_rf = RandomForestRegressor()
model_rf.fit(X_train, Y_train)
model_lr = LinearRegression()
model_lr.fit(X_train, Y_train)
model_gb = GradientBoostingRegressor()
model_gb.fit(X_train, Y_train)
model_et=ExtraTreesRegressor()
model_et.fit(X_train,Y_train)
Y_pred_rf=model_rf.predict(X_test)
Y_pred_lr=model_lr.predict(X_test)
Y_pred_gb=model_gb.predict(X_test)
Y_pred_et=model_et.predict(X_test)


In [ ]:
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error


def evaluvate_model(Y_test,Y_pred,model_name):
    mae = mean_absolute_error(Y_test, Y_pred)
    mse = mean_squared_error(Y_test, Y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(Y_test, Y_pred)
    print(f"\n{model_name} performance")
    print(f"MAE:{mae:2f}")
    print(f"MSE:{mse:2f}")
    print(f"RMSE:{rmse:2f}")
    print(f"R2:{r2:4f}")
evaluvate_model(Y_test,Y_pred_rf,"Random Forest")
evaluvate_model(Y_test,Y_pred_lr,"Linear Regressor")
evaluvate_model(Y_test,Y_pred_gb,"Gradian booster")
evaluvate_model(Y_test,Y_pred_et,"Extra Tree")





In [ ]:
with open("extratree_reg_model_v2.pkl","wb")as f:
    pickle.dump(model_et,f)

In [ ]:
df["status"]

In [ ]:
#df["status"].isin(["draft"]).any()

In [ ]:
df = df[df["status"].isin(["Won", "Lost"])].copy()
df["status"] = df["status"].cat.remove_unused_categories()


In [ ]:
df["status"]

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df["status_encoded"] = le.fit_transform(df["status"])


In [ ]:
df["status_encoded"]

In [ ]:
from sklearn .model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier

x_clf=df_final
y_clf=df["status_encoded"]
X_clf_train,X_clf_test,Y_clf_train,Y_clf_test=train_test_split(
    x_clf,y_clf,test_size=0.2,random_state=42)
dt_clf = DecisionTreeClassifier(max_depth=10, random_state=42)
rf_clf = RandomForestClassifier(n_estimators=200, max_depth=15, random_state=42)
et_clf = ExtraTreesClassifier(n_estimators=200, max_depth=15, random_state=42)
log_clf = LogisticRegression(max_iter=1000)
log_clf.fit(X_clf_train, Y_clf_train)
dt_clf.fit(X_clf_train, Y_clf_train)
rf_clf.fit(X_clf_train,Y_clf_train)
et_clf.fit(X_clf_train,Y_clf_train)

Y_pred_dt=dt_clf.predict(X_clf_test)
Y_pred_rf=rf_clf.predict(X_clf_test)
Y_pred_et=et_clf.predict(X_clf_test)
Y_pred_log=log_clf.predict(X_clf_test)


In [ ]:
from sklearn.metrics import accuracy_score
def accuracy(Y_clf_test,Y_pred,model_name):
    acc_score=accuracy_score(Y_clf_test,Y_pred)
    print(f"{model_name}Accuracy")
    print(f"Accuracy,{acc_score:.4f}")
accuracy(Y_clf_test,Y_pred_dt,"Decition Tree")
accuracy(Y_clf_test,Y_pred_rf,"Random Forest")
accuracy(Y_clf_test,Y_pred_et,"Extra Tree")
accuracy(Y_clf_test,Y_pred_log,"Logistic Regressor")




In [ ]:
with open("random_forest_clas_model_v2.pkl","wb")as f:
    pickle.dump(rf_clf,f)

In [ ]:
#checkiing user input 
user_input = {
    "quantity tons": 25,
    "thickness": 5,
    "delivery_days": 7  # calculated from delivery date - item date
}
user_df = pd.DataFrame([user_input])
user_df

In [ ]:
user_log = np.log1p(user_df[num_col])
user_log

In [ ]:
with open("scaler.pkl", "rb") as file:
    saved = pickle.load(file)
#user_scaler = saved["scaler"]
#num_col = saved["num_col"] 
user_scaled = saved.transform(user_log)
print(user_scaled)

In [ ]:
num_col.append("delivery_days")
#num_col.remove("delivery_days")

In [ ]:
num_col

In [ ]:
#pickling numeric scalled value
import pickle
with open("scaler.pkl","wb") as file:
    pickle.dump(scaler,file)
          
          
    


In [ ]:
#pickle the ordinal numeric
bin_info={"bins":[0, 800, 1000, 1200, 1500, 2000,3000],
                         "labels":["very_small", "small", "medium", "large", "very_large","extra_large"]
}
with open("width_binning.pkl","wb") as file:
    pickle.dump(bin_info,file)

In [ ]:
#pickle of the ordinal encoding 
with open("width_encoder.pkl","wb") as file:
    pickle.dump(encoder,file)

In [ ]:
print(saved.keys())

In [ ]:
#checking width bin
user_width=900
with open("width_binning.pkl","rb") as file:
    saved=pickle.load(file)
#print(saved.keys())
width_bins=saved["bins"]
width_lable=saved["labels"]
user_bin=np.digitize(user_width,width_bins)
user_lable=width_lable[user_bin-1]
print(user_bin)
print(user_lable)
#with open("width_encoder.pkl","rb")as file:
  #  en=pickle.load(file)
#user_en=bin.transform(user_width)

In [ ]:
print(user_bin.dtype)

In [ ]:
#user_bin_int=int(user_bin)
with open("width_encoder.pkl","rb")as file:
    en=pickle.load(file)
user_width_df=pd.DataFrame({"width_bin":[user_lable]})
user_en=en.transform(user_width_df)
print(user_en)

In [ ]:
df["width"]

In [ ]:
#for the width column we apply binning and encoding

with open("width_binning.pkl","rb") as file:
    saved=pickle.load(file)
width_bins=saved["bins"]
width_lable=np.array(saved["labels"])
width_bin_col_idx=np.digitize(df["width"],width_bins)
width_lable_col=width_lable[(width_bin_col_idx-1)]
print(width_lable_col)
#print(width_bin_col_idx)



    

In [ ]:
#user_bin_int=int(user_bin)
with open("width_encoder.pkl","rb")as file:
    en=pickle.load(file)
width_label_col_df=pd.DataFrame({"width_bin":width_lable_col})
width_en=en.transform(width_label_col_df)
print(width_en)

In [ ]:
width_en_df=pd.DataFrame({"width_encoded":width_en.ravel()})

In [ ]:
len(ohe_df)

In [ ]:
len(num_df)

In [ ]:
len(width_en_df)

In [ ]:
df_final=df_fin = pd.concat([width_en_df.reset_index(drop=True),
                    num_df.reset_index(drop=True),
                    ohe_df.reset_index(drop=True)], axis=1)


In [ ]:
df_final

In [ ]:
x

In [ ]:
x=df_final.copy()

In [ ]:
x

In [ ]:
x=x.drop(columns="width_bin")

In [ ]:
del x
x

In [ ]:
y=df["selling_price"]

In [ ]:
y

In [ ]:
X_train

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
model_rf = RandomForestRegressor()
model_rf.fit(X_train, Y_train)
model_lr = LinearRegression()
model_lr.fit(X_train, Y_train)
model_gb = GradientBoostingRegressor()
model_gb.fit(X_train, Y_train)
Y_pred_rf=model_rf.predict(X_test)
Y_pred_lr=model_lr.predict(X_test)
Y_pred_gb=model_gb.predict(X_test)


In [ ]:
Y_pred_rf

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

RF_mae = mean_absolute_error(Y_test, Y_pred_rf)
RF_mse = mean_squared_error(Y_test, Y_pred_rf)
RF_rmse = np.sqrt(RF_mse)
RF_r2 = r2_score(Y_test, Y_pred_rf)
LR_mae = mean_absolute_error(Y_test, Y_pred_lr)
LR_mse = mean_squared_error(Y_test, Y_pred_lr)
LR_rmse = np.sqrt(LR_mse)
LR_r2 = r2_score(Y_test, Y_pred_lr)
GB_mae = mean_absolute_error(Y_test, Y_pred_gb)
GB_mse = mean_squared_error(Y_test, Y_pred_gb)
GB_rmse = np.sqrt(GB_mse)
GB_r2 = r2_score(Y_test, Y_pred_rf)


print("RF_MAE:", RF_mae)
print("RF_RMSE:", RF_rmse)
print("RF_R² Score:", RF_r2)
print("LR_MAE:", LR_mae)
print("LR_RMSE:", LR_rmse)
print("LR_R² Score:", LR_r2)
print("GB_MAE:", GB_mae)
print("GB_RMSE:", GB_rmse)
print("GB_R² Score:", GB_r2)


In [ ]:
import pickle
with open("model_copper_rfr.pkl", "wb") as file:
    pickle.dump(model_rf, file)

In [ ]:
with open("model_copper_rfr.pkl", "rb") as file:
    loaded_model_rfr=pickle.load(file)

In [ ]:
y_pred_rfr=loaded_model_rfr.predict(X_test)

In [ ]:
MAE: 51.2508503049975
RMSE: 79.01782239107797
R² Score: 0.8366496721030081

In [ ]:
user_width_df

In [ ]:
#scalling for quantity tons
df["quantity tons"]


In [ ]:
#AFTER CAPPING STILL IT HAS RIGHT SKEWED 
# SO LOG TRANSFER MATION IS USED TO MAKE NORMAL DISTRIBUTION
#✔ If using Linear Regression, Ridge, Lasso, KNN → StandardScaler
sns.histplot(x="quantity tons",data=df)
plt.show()

In [ ]:
#APPLY LOG TRANSFORM it s used for create normal distribution
df["quantity tons"]=np.log1p(df["quantity tons"])

In [ ]:
#df["quantity tons"]=pd.to_numeric(df["quantity tons"],errors="coerce")

In [ ]:
df["quantity tons"].dtype

In [ ]:
sns.histplot(x="quantity tons",data=df)

In [ ]:
#SCALING 
# IM USING STANDERDISED SCALLER
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df["quantity_scaled"] = scaler.fit_transform(df[["quantity tons"]])


In [ ]:
#plot distribution 
sns.histplot(x="thickness",data=df)

In [ ]:
df["thickness"].mean(), df["thickness"].median()
# if mean >median 
#2.6>1.5
#so it s highly right skewed
#use log transfermation for reduce skewness

In [ ]:
df["thickness_log"]=np.log1p(df["thickness"])
df["thickness_log"]

In [ ]:
df["thickness_log"]=np.log1p(df["thickness"])
df["thickness_log"]

In [ ]:
#plot distribution 
sns.histplot(x="thickness_log",data=df)

In [ ]:
#for regressitoin use staderdised scaller
from sklearn.preprocessing  import StandardScaler
scaler=StandardScaler()
df["thickness_scaled"] = scaler.fit_transform(df[["thickness_log"]])


In [ ]:
#here im using binning because width hist plot is multi model distribution
#WHY BINNING 
#Multimodal structure automatically handled
#✔ Regression model-க்கு stability அதிகரிக்கும்
#✔ Outliers automatically compressed
#✔ Industry application-ல் width mostly category-like (width standards)

df["width_bin"] = pd.cut(df["width"],
                         bins=bin_info["bins"],
                         labels=bin_info["labels"])


In [ ]:
df["width_bin"]

In [ ]:
#encoding width column
# we dont use lable encoder because it gives number in alphabetical order
#use ORDINAL ENCODING
# encoding after extending the lable 
from sklearn.preprocessing import OrdinalEncoder
order=[["very_small", "small", "medium", "large", "very_large","extra_large"]]
encoder=OrdinalEncoder(categories=order)
df["width_encoder"]=encoder.fit_transform(df[["width_bin"]])

In [ ]:
df["width_encoder"]

In [ ]:
ohe_array

In [ ]:
df["country"].info()

In [ ]:
ohe_array

In [ ]:
#after encoding we need to convert it as the  dataframe
ohe_df=pd.DataFrame(ohe_array,columns=ohe_encoding.get_feature_names_out(cat_cols))
ohe_df

In [ ]:
df_fine=pd.concat([df_fine,ohe_df,])

In [ ]:
#country column already in num but it doesnot provide meaning full information 
# so we need to convert this into a categorical column and apply one hot encoding
df["country"]=df["country"].astype(str)

In [ ]:
#apply one hot encoding
country_dummies=pd.get_dummies(df["country"],prefix="country")
#pd.concat([df,dummies],axis=1)

In [ ]:
df["country"]

In [ ]:
country_dummies.columns

In [ ]:
country_dummies=country_dummies.astype(int)
country_dummies

In [ ]:
df=pd.concat([df,country_dummies],axis=1)

In [ ]:
df["customer"].nunique()

In [ ]:
df.describe()

In [ ]:
#customer is an id column so we donot use that column for ML
#here the status is not used as a input feature  
# because staus is Won / Lost” என்பது order முடிந்த பிறகு தெரியு


In [ ]:
df["application"].nunique()

In [ ]:
df["item type"].str.upper()

In [ ]:
item_dummies=pd.get_dummies(df["item type"],prefix="item")

In [ ]:
df.columns

In [ ]:
item_dummies=item_dummies.astype(int)

In [ ]:

item_dummies.columns=item_dummies.columns.str.upper()

In [ ]:
item_dummies

In [ ]:
df

In [ ]:
df=pd.concat([df,item_dummies],axis=1)

In [ ]:
#actually application is a categorical column so you dont need to find  outlier
#its alredy in the encoded form

df["application"].nunique()

In [ ]:
#use one hot encoding because its not order or level
application_dummies=pd.get_dummies(df["application"],prefix="application").astype(int)

In [ ]:
application_dummies

In [ ]:
df=pd.concat([df,application_dummies],axis=1)

In [ ]:
df["product_ref"].nunique()

In [ ]:
#use one hot encoding for the product ref
prod_dummies=pd.get_dummies(df["product_ref"],prefix="product").astype(int)

In [ ]:
df.columns

In [ ]:
df["delivery_days"]=(df["delivery date"]-df["item_date"]).dt.days

In [ ]:
df["delivery_days"].dtype

In [ ]:
sns.histplot(x="delivery_days",data=df)

In [ ]:
sns.boxplot(x="delivery_days",data=df)

In [ ]:
Q1 = df['delivery_days'].quantile(0.25)
Q3 = df['delivery_days'].quantile(0.75)
IQR = Q3 - Q1

lower_limit = Q1 - 1.5 * IQR
upper_limit = Q3 + 1.5 * IQR

outliers = df[(df['delivery_days'] < lower_limit) | (df['delivery_days'] > upper_limit)]
outliers.shape


In [ ]:
# cappling
df["delivery_days"]=df["delivery_days"].clip(lower_limit,upper_limit)

In [ ]:
sns.boxplot(x="delivery_days",data=df)

In [ ]:
# use staderdised scallar 
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df["delivery_days_scaled"] = scaler.fit_transform(df[["delivery_days"]])


In [ ]:
import matplotlib.pyplot as plt
num_column=["quantity tons","application","width","thickness","selling_price","product_ref"]
df_num=df[num_column]
corr=df_num.corr()
plt.figure(figsize=(8, 6))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Heatmap")
plt.show()

In [ ]:
final_features=

In [ ]:
df

In [ ]:
df["selling_price"].mean()

In [ ]:
df.columns

In [ ]:
#feature_sellection
final_features = [
    "quantity_scaled",
    
    "width_encoder",
    "thickness_scaled",
    "delivery_days_scaled",
    
    # encoded columns
] + [col for col in df.columns if col.startswith("country_")] \
  + [col for col in df.columns if col.startswith("item_")]\
  +[col for col in df.columns if col.startswith("application_")]


In [ ]:
df_final=df[final_features]

In [ ]:
df_final=df_final.drop(columns="item_date")

In [ ]:
df_final.columns

In [ ]:
#x=x.drop(columns="item_date")

In [ ]:
#col_drop=[col for col in df_final.columns if col.startswith("country_")]

In [ ]:
#col_drop_item=[col for col in df_final.columns if col.startswith("item_")]

In [ ]:
#df_final=df_final.drop(columns=col_drop)

In [ ]:
#df=df.drop(columns=col_drop)

In [ ]:
#df_final=df_final.drop(columns=col_drop_item)

In [ ]:
#df=df.drop(columns=col_drop_item)

In [ ]:
df_final

In [ ]:
[col for col in df_final.columns if col.startswith("item_")]

In [ ]:
x=df_final.copy()
x

In [ ]:
y=df["selling_price"]
y

In [ ]:
from sklearn .model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(
    x,y,test_size=0.2,random_state=42)

In [ ]:
X_test

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()


In [ ]:
model_=model.fit(X_train, Y_train)

In [ ]:
y_pred=model_.predict(X_test)

In [ ]:
y_pred

In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score
import numpy as np
#evaluation=mean_absolute_error(Y_test,y_pred)
r2=r2_score(Y_test,y_pred)

In [ ]:
r2

In [ ]:
evaluation

In [ ]:
df_final.isnull().sum()

In [ ]:
#df["thickness_scaled"].isnull().sum()
#df["thickness"]=df.drop()

In [ ]:
#df = df.dropna(subset=["thickness"])


In [ ]:
from sklearn.ensemble import RandomForestRegressor
model_rf = RandomForestRegressor()

In [ ]:
model_rf=model_rf.fit(X_train, Y_train)

In [ ]:
Y_pred_rf=model_rf.predict(X_test)

In [ ]:
Y_pred_rf

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

mae = mean_absolute_error(Y_test, Y_pred_rf)
mse = mean_squared_error(Y_test, Y_pred_rf)
rmse = np.sqrt(mse)
r2 = r2_score(Y_test, Y_pred_rf)

print("MAE:", mae)
print("RMSE:", rmse)
print("R² Score:", r2)


In [ ]:
#by using ranndom forest regressor 
#MAE%=(51.22/823.82)​×100=6.2% error
#Anything < 10% error → Good

#Anything < 5% error → Very Good

#Anything < 2% error → Excellent

#Your model is close to ve
#6.2% is good 

In [ ]:
#Average selling price = 823.82


#RMSE = 79.05


#Now calculate percentage error:
#(79.05/823.82)×100= 9.59%


#🎯 Conclusion About Your RMSE
#⭐ RMSE = 79
#⭐ Average = 823
#⭐ Error = 9.6%
#👉 This is GOOD.
#👉 Anything below 10% is considered a good RMSE for regression.

#💬 Why RMSE looks bigger than MAE?
#Because:


#MAE → Average error (normal mistake)


#RMSE → Punishes big mistakes more
#→ So RMSE is always higher than MAE


#This is normal and expected.

#📝 Tamil Explanation (தமிழில்)
#உங்கள் selling price சராசரி = 823
#RMSE = 79
#Percentage error:
#79/823=9.6%79 / 823 = 9.6\%79/823=9.6%
#👉 இது நல்ல value.
#👉 10% க்குக் கீழே இருக்கிறது — அதனால் model நல்லா predict செய்கிறது.

#If you want even better accuracy
#I can tell you how to tune RandomForest or try XGBoost.

In [ ]:
#CLASSIFICATION PROBLEM
df_final.columns

In [ ]:
df["status"].unique()

In [ ]:
df["status"].value_counts()

In [ ]:
df["status"].info()

In [ ]:
#to remove unused values
df["status"] = df["status"].cat.remove_unused_categories()
df["status"].cat.categories


In [ ]:
df["status"].value_counts()

In [ ]:
#df["status"].unique()

In [ ]:
df["status"].cat.categories

In [ ]:
#df[df["status"].isin(["Won","Lost"])]

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Initialize encoder
le = LabelEncoder()

# Fit and transform the target
y_c = le.fit_transform(df["status"])


In [ ]:
le.classes_

In [ ]:
y_c

In [ ]:
x_c

In [ ]:
x_c=df_final.copy()

In [ ]:
df_final=df_final.drop(columns="item_date")

In [ ]:
df_final.columns

In [ ]:
from sklearn.model_selection import train_test_split
#c_moldel=train_test_split()
X_C_train,X_C_test,Y_C_train,Y_C_test=train_test_split(
    x_c,y_c,test_size=0.2,random_state=42)

In [ ]:
X_C_train

In [ ]:
X_C_test

In [ ]:
from sklearn.tree import DecisionTreeClassifier
model_dt=DecisionTreeClassifier()
model_dt.fit(X_C_train,Y_C_train)


In [ ]:
y_c_pred=model_dt.predict(X_C_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(Y_C_test,y_c_pred)

In [ ]:
accuracy

In [ ]:
# non linear so use randomforest 
from sklearn.ensemble import RandomForestClassifier
model_rfc=RandomForestClassifier()
model_rfc.fit(X_C_train,Y_C_train)


In [ ]:
y_rfc_pred=model_rfc.predict(X_C_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_rfc=accuracy_score(Y_C_test,y_rfc_pred)

In [ ]:
accuracy_rfc

In [ ]:
import pickle
with open("model_copper_rfr.pkl", "wb") as file:
    pickle.dump(model_rf, file)

In [ ]:
with open("model_copper_rfr.pkl", "rb") as file:
    loaded_model_rfr=pickle.load(file)

In [ ]:
y_pred_rfr=loaded_model_rfr.predict(X_test)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

mae = mean_absolute_error(Y_test, y_pred_rfr)
mse = mean_squared_error(Y_test, y_pred_rfr)
rmse = np.sqrt(mse)
r2 = r2_score(Y_test, y_pred_rfr)

print("MAE:", mae)
print("RMSE:", rmse)
print("R² Score:", r2)


In [ ]:
with open("model_copper_rfc.pkl", "wb") as file:
    pickle.dump(model_rfc, file)

In [ ]:
with open("model_copper_rfc.pkl", "rb") as file:
    loaded_model_rfc=pickle.load(file)

In [ ]:
y_pred_rfc=loaded_model_rfc.predict(X_C_test)

In [ ]:
from sklearn.metrics import accuracy_score
loaded_accuracy_rfc=accuracy_score(Y_C_test,y_pred_rfc)

In [ ]:
loaded_accuracy_rfc

In [ ]:
def user(input()):
    while True:
        try:

            user_quant=float(input("Enter quantity"))
            print(quant)
            width = float(input("width"))
            thickness = float(input("thickness"))
            item_date=float(input("item_date"))
            country = float(input("country"))
            delivery_date =float(input("delivery_date"))
            application = float(input("application"))
            break
        except:
            print("enter numeric values only")
    print(width)
    print(thickness)
    print(item_date)
    print(country)
    print(delivery_date)
    print(application)
    return width,thickness,item_date,country,delivery_date,application


In [ ]:
#if user_quant<0:
#ஒவ்வொரு user input-க்கும் fit செய்தால்:

#Mean மாற்றும்

#Standard deviation மாற்றும்

#Training scale உடைந்துவிடும்

#Prediction தவறாக வரும்
user_quant=float(input())
qty_user_capped = max(q_lower_limit, min(user_quant, q_upper_limit))
print(qty_user_capped)
user_quant_log = np.log1p(qty_user_capped).reshape(1,1)
print(user_quant_log )
user_quantity_scaled = scaler.transform(user_quant_log)

print(user_quantity_scaled)

In [ ]:
user_thickness=float(input())
thickness_user_capped = max(t_lower_limit, min(user_thickness, t_upper_limit))
print(thickness_user_capped)
user_thickness_log = np.log1p(thickness_user_capped).reshape(1,1)
print(user_thickness_log )
user_thickness_scaled = scaler.transform(user_thickness_log)

print(user_thickness_scaled)


In [ ]:
user_width=float(input())
w_lowerlimit=1.0
w_upper_limit=3000
width_user_capping=max(w_lowerlimit,min(user_width,w_upper_limit))
user_width_bin= pd.cut([width_user_capping],
                         bins=[0, 800, 1000, 1200, 1500, 2000,3000],
                         labels=["very_small", "small", "medium", "large", "very_large","extra_large"])[0]
user_width_encoded = encoder.transform(pd.DataFrame({"width_bin":[user_width_bin]}))[0][0]
user_width_encoded

In [ ]:
df["width"].max()

In [ ]:
user_country=float(input())
print(user_country)
user_df=pd.DataFrame([user_country],columns=["country"])
user_df
user_dummy=pd.get_dummies(user_df["country"],prefix="country").astype(int)
# get country_dummies.columns
print(user_dummy)
coun_dummy_col=country_dummies.columns
#print(len(country_dummies))
user_dummy_reindex=user_dummy.reindex(columns=coun_dummy_col,fill_value=0)
user_dummy_reindex

In [ ]:
user_item=input().upper()
print(user_item)
user_item_df=pd.DataFrame([user_item],columns=["item"])
user_item_df
user_item_dummy=pd.get_dummies(user_item_df,prefix="ITEM").astype(int)
print(user_item_dummy)
item_cols=item_dummies.columns
w
user_item_reindex=user_item_dummy.reindex(columns=item_cols,fill_value=0)
print(user_item_reindex)

In [ ]:
user_application=float(input())
print(user_application)
user_app_df=pd.DataFrame([user_application],columns=["application"])
user_app_df
user_app_dummy=pd.get_dummies(user_app_df,prefix="application").astype(int)
print(user_app_dummy)
app_cols=application_dummies.columns

user_app_reindex=user_app_dummy.reindex(columns=app_cols,fill_value=0)
print(user_app_reindex)

In [ ]:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181673 entries, 0 to 181672
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   id             181671 non-null  object 
 1   item_date      181672 non-null  float64
 2   quantity tons  181673 non-null  object 
 3   customer       181672 non-null  float64
 4   country        181645 non-null  float64
 5   status         181671 non-null  object 
 6   item type      181673 non-null  object 
 7   application    181649 non-null  float64
 8   thickness      181672 non-null  float64
 9   width          181673 non-null  float64
 10  material_ref   103754 non-null  object 
 11  product_ref    181673 non-null  int64  
 12  delivery date  181672 non-null  float64
 13  selling_price  181672 non-null  float64
dtypes: float64(8), int64(1), object(5)
memory usage: 19.4+ MB